In [6]:
import subprocess
from sage.cboot.scalar import context_for_scalar

In [7]:
# Define the context variable for the bootstrap problems.
# Lambda controls the maximal number of derivatives acting on conformal blocks.
# epsilon is (d-2)/2, so 0 for 2d, 0.5 for 3d, and 1 for 4d.
# nMax is the expansion parameter for scala 
context=context_for_scalar(Lambda=15,epsilon=0.5,nMax=250)

In [8]:
def fixed_spin_contributuion(delta, spin,shift,nu_max=12):
    # defines a data to approxmate a conformal block of given spin. nu_max is the cutoff for the included pole.
    # The last two parameters are to be used for 4pt functions with non-identical dimensions
    q=context.rational_approx_data(nu_max,spin,0,0)
    
    # Compute the approximation
    approx=q.approx()
    
    # Shift by the unitarity bound / hypothetical gap
    approx=approx.shift(shift)
    
    # Apply the matrix which represents v^{\Delta _ \phi}-multplication and anti-symmetrization
    F_body=context.make_F_minus_matrix(delta).dot(approx.matrix)
    return context.prefactor_numerator(approx.prefactor,F_body.reshape((1,1,len(F_body))), )

def unitarity_bounds(lmax,modify_list):
    ubs=dict([(x,x+2*context.epsilon) for x in range(0,lmax+1,2)])
    updates=dict([(x[0],context(x[1])) for x in modify_list.items()])
    ubs.update(updates)
    return ubs

def ising_singlet_bound_SDP(delta,modify_list,lmax=30,nu_max=12):
    delta=context(delta)
    ubs=unitarity_bounds(lmax,modify_list)
    polynomial_Vector_Matrices=[fixed_spin_contributuion(delta,x,ubs[x],nu_max=nu_max) for x in ubs]
    normalization=context.make_F_minus_matrix(delta).dot(context.identity_vector())
    obj=normalization*0
    return context.SDP(normalization,obj,polynomial_Vector_Matrices)

In [9]:
m=ising_singlet_bound_SDP(0.518,{0:1.2})

In [11]:
subprocess.Popen(["sdpb","-s","test.xml","--findPrimalFeasible","--findDualFeasible"],\
                 stderr=subprocess.PIPE,stdout=subprocess.PIPE).communicate()

OSError: [Errno 2] No such file or directory